In [4]:
from strategy.match_spy import MatchSpy
from strategy.utils import HistoricalObservationSetCursor
from strategy.features.base.kalshi import daily_spy_range_kalshi_markets
from strategy.features.base.spy import hist_spy_feature
from strategy.sim.active_ioc import ActiveIOCStrategySimulator
from data.coledb.coledb import ColeDBInterface, ColeDBCursor

In [5]:
import pathlib
import datetime

from helpers.types.money import Price
from helpers.types.orders import Quantity

es_file = pathlib.Path("./es_data/sep.csv")
# I need a day both with ES and with kalshi INXD data. 
# We, surprisingly, don't have one :(. 

date = datetime.date(year=2023, month=9, day=12)
day_start = datetime.datetime.combine(date=date, time=datetime.time.min)
day_end = datetime.datetime.combine(date=date, time=datetime.time.max)

spy_cursor = hist_spy_feature(es_file=es_file)
kalshi_spy_markets = daily_spy_range_kalshi_markets(date=date)
for m in kalshi_spy_markets:
    strategy = MatchSpy(spy_source=spy_cursor, kalshi_spy_markets=kalshi_spy_markets, price=Price(10), qty=Quantity(1))
    historical_features = HistoricalObservationSetCursor.from_observation_streams(feature_streams=[spy_cursor])
    kalshi_orderbook_updates = ColeDBInterface().read_cursor(ticker=m.ticker, start_ts=day_start, end_ts=day_end)
    sim = ActiveIOCStrategySimulator(kalshi_orderbook_updates=kalshi_orderbook_updates, historical_data=historical_features)
    result = sim.run(strategy=strategy)
    print(result.pnl)

FileNotFoundError: [Errno 2] No such file or directory: 'src/data/coledb/storage/INXD/23SEP12'